In [1]:
import pandas as pd
news_data = pd.read_csv("./wiki_data/sus_reports_url_top3.csv")
news_data

,id,Unnamed: 0,Unnamed: 0.1,site,summary,url,rank,company,relevance_score
0,276,0,0,Research: Rating Action: Moody's changes 10117...,Corporate governance risks at 1011778 B.C. is ...,https://www.moodys.com/research/Moodys-changes...,9340,1011778 BC ULC,1.500872
1,278,2,2,Research: Rating Action: Moody's downgrades Ta...,"sustainability. diversity, equity & inclusion....",https://www.moodys.com/research/Moodys-downgra...,11179,1011778 BC ULC,1.591902
2,277,1,1,"Research: Rating Action: Moody's places 1,721 ...","3. To the extent permitted by law, Moody’s and...",https://www.moodys.com/research/Moodys-places-...,12366,1011778 BC ULC,1.643596
3,280,4,1,"3D Systems Announces Launch of Environmental, ...","3D Systems Announces Launch of Environmental, ...",https://www.3dsystems.com/press-releases/3d-sy...,6748,3D Systems Corp,1.323604
4,287,11,8,3D Systems Reports 2012 Results | 3D Systems,3D Systems Reports 2012 Results. Delivers Reco...,https://www.3dsystems.com/press-releases/3d-sy...,9084,3D Systems Corp,1.486445
...,...,...,...,...,...,...,...,...,...
5603,15046,14955,0,Sustainability - thyssenkrupp,"thyssenkrupp is committed to open, constructiv...",https://www.thyssenkrupp.com/en/company/sustai...,4708,thyssenkrupp AG,1.185300
5604,15049,14958,3,Dialogue & Commitment - thyssenkrupp,thyssenkrupp regards the ideas and requirement...,https://www.thyssenkrupp.com/en/company/sustai...,5053,thyssenkrupp AG,1.207290
5605,15596,15525,8,Sustainability | u-blox,Environmental responsibility. u‑blox has a lon...,https://www.u-blox.com/en/sustainability-3,1439,u-blox Holding AG,0.997414
5606,15592,15521,4,Sustainability | u-blox,No business operates in isolation: for u‑blox ...,https://www.u-blox.com/en/sustainability-2,1615,u-blox Holding AG,1.009597


In [2]:
# news_data = news_data[news_data["rank"]<=3]

In [5]:
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
url = "https://www.3m.com/3M/en_US/sustainability-us/annual-report/"
response = requests.get(url, timeout=20)
soup= BeautifulSoup(response.text, "html.parser")  
pdf_list = []
for link in soup.select("a[href$='.pdf']"):
    pdf_list.append(urljoin(url,link['href']))

In [ ]:
pdf_list

In [25]:
df = pd.read_csv("./wiki_data/sus_reports_url.csv")
len(df[df.url.str.contains(".pdf")].company.unique())
df[df.url.str.contains(".pdf")].url.values
df[df.company=="Adobe Inc"].url.values

array(['https://www.adobe.com/corporate-responsibility/sustainability.html',
       'https://www.adobe.com/corporate-responsibility/corporate-responsibility-reports.html',
       'https://www.adobe.com/corporate-responsibility/sustainability/corporate-policy.html',
       'https://www.adobe.com/content/dam/cc/en/investor-relations/pdfs/Governance-and-Sustainability-Committee-Charter.pdf',
       'https://www.adobe.com/content/dam/cc/en/corporate-responsibility/pdfs/Adobe-CSR-Report-FY2019.pdf',
       'https://www.adobe.com/uk/corporate-responsibility/corporate-responsibility-reports.html',
       'https://www.adobe.com/corporate-responsibility/awards.html',
       'https://www.adobe.com/corporate-responsibility.html',
       'https://www.adobe.com/corporate-responsibility/sustainability/energy-conservation.html'],
      dtype=object)

In [1]:
import sys
sys.path.append("/home/qingzhi/.conda/envs/test/lib/python3.9/site-packages")

In [30]:
import pandas as pd
news_data = pd.read_csv("./wiki_data/sus_reports_url_top3.csv")
news_data = news_data[~news_data.url.str.contains(".pdf")]

In [33]:


import grequests

class Test(object):
    def __init__(self, urls):
        self.urls = urls

    def exception(self, request, exception):
        print("Problem: {}: {}".format(request.url, exception))

    def async1(self):
        results = grequests.map((grequests.get(u, timeout=5, stream=False) for u in self.urls), exception_handler=self.exception, size=20)
        # print(results)
        return results
    
# test = Test(news_data.url.values.tolist()[0:100])
# results = test.async1()
# results

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.parse import urljoin
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

content_list = []
for i in tqdm(list(chunks(range(0, len(news_data.url.values)), 20))):
    urls = news_data.url.values[i]
    test = Test(urls)
    results = test.async1()
    # for req, url in tqdm(zip(results, urls), total=len(urls), leave=False):
    for req, url in tqdm(zip(results, urls), total=len(urls), leave=False):
        if req == None:
            content_list.append(None)
        elif req.status_code == 200:
            soup = BeautifulSoup(req.content, 'html.parser', from_encoding="iso-8859-1")
            req.close()
            pdf_list = []
            for link in soup.select("a[href$='.pdf']"):
                pdf_list.append(urljoin(url,link['href']))
            content_list.append(pdf_list)
        else:
            content_list.append(None)
            
# news_data["pdf_list"] = content_list
# content_list

  0%|                                                   | 0/219 [00:00<?, ?it/s]

Problem: https://www.3m.com/3M/en_US/sustainability-us/annual-report/: HTTPSConnectionPool(host='www.3m.com', port=443): Read timed out. (read timeout=5)
Problem: https://www.3m.com/3M/en_US/sustainability-us/annual-report/past-reports-policies/: HTTPSConnectionPool(host='www.3m.com', port=443): Read timed out. (read timeout=5)
Problem: https://www.3m.com/3M/en_US/sustainability-us/annual-report/past-reports-policies/sustainable-forestry/: HTTPSConnectionPool(host='www.3m.com', port=443): Read timed out. (read timeout=5)
Problem: https://www.abnamro.com/en/news/abn-amro-issues-climate-publication-guiding-a-banks-portfolio-to-paris: HTTPSConnectionPool(host='www.abnamro.com', port=443): Read timed out. (read timeout=5)
Problem: https://www.abnamro.com/nl/nieuws/abn-amro-lanceert-sustainable-impact-fund: HTTPSConnectionPool(host='www.abnamro.com', port=443): Read timed out. (read timeout=5)
Problem: https://www.abnamro.com/en/news/abn-amro-launches-sustainable-impact-fund: HTTPSConnectio


  0%|▏                                          | 1/219 [00:10<39:06, 10.76s/it]

Problem: https://adler-ag.com/en/sustainability/: HTTPSConnectionPool(host='adler-ag.com', port=443): Max retries exceeded with url: /en/sustainability/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)')))
Problem: https://adler-ag.com/en/investor-relations/publications/financial-reports-2/: HTTPSConnectionPool(host='adler-ag.com', port=443): Max retries exceeded with url: /en/investor-relations/publications/financial-reports-2/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)')))



  1%|▍                                          | 2/219 [00:12<19:54,  5.50s/it]

...


 99%|████████████████████████████████████████▍| 216/219 [18:01<00:14,  4.82s/it]

Problem: https://www.yum.com/wps/portal/yumbrands/Yumbrands/news/press-releases/yum+brands+annual+global+citizenship+sustainability+report+showcases+companys+commitment+to+socially+responsible+growth+highlights+progress+around+priority+areas: HTTPSConnectionPool(host='www.yum.com', port=443): Read timed out. (read timeout=5)
Problem: https://www.yum.com/wps/portal/yumbrands/Yumbrands/news/press-releases/yum+brands+annual+global+citizenship+and+sustainability+report+showcases+companys+strategic+investment+in+socially+responsible+growth: HTTPSConnectionPool(host='www.yum.com', port=443): Read timed out. (read timeout=5)
Problem: https://www.yum.com/wps/portal/yumbrands/Yumbrands/news/company-stories-article/Yum+Brands+advances+its+sustainability+journey+through+transparent+reporting+to+CDP: HTTPSConnectionPool(host='www.yum.com', port=443): Read timed out. (read timeout=5)



 99%|████████████████████████████████████████▋| 217/219 [18:07<00:10,  5.05s/it]

Problem: https://www.adidas-group.com/en/sustainability/reporting/sustainability-reports/: HTTPSConnectionPool(host='www.adidas-group.com', port=443): Read timed out. (read timeout=5)
Problem: https://www.adidas-group.com/en/sustainability/managing-sustainability/environmental-approach/: HTTPSConnectionPool(host='www.adidas-group.com', port=443): Read timed out. (read timeout=5)
Problem: https://www.adidas-group.com/en/sustainability/reporting/green-company/: HTTPSConnectionPool(host='www.adidas-group.com', port=443): Read timed out. (read timeout=5)



100%|█████████████████████████████████████████| 219/219 [18:16<00:00,  5.01s/it]


In [36]:
news_data["pdf_url"] = content_list
news_data.to_csv("./wiki_data/sus_reports_pdf_url.csv")

In [37]:
news_data = news_data.explode("pdf_url")


In [45]:
news_data = news_data.dropna()
# news_data

In [20]:
df = pd.read_csv("./wiki_data/sus_reports_url.csv")
df2 = df[df.url.str.lower().str.contains("pdf")].rename(columns = {"url":"pdf_url"})

In [29]:
from ast import literal_eval

news_data = pd.read_csv("./wiki_data/sus_reports_pdf_url.csv")
news_data = news_data.dropna()
news_data["pdf_url"] = news_data["pdf_url"].apply(literal_eval)
news_data = news_data.explode("pdf_url")
news_data = news_data.dropna()
news_data.drop("url",axis=1,inplace=True)
news_data

,Unnamed: 0,id,Unnamed: 0.1,Unnamed: 0.1.1,site,summary,rank,company,relevance_score,pdf_url
0,0,276,0,0,Research: Rating Action: Moody's changes 10117...,Corporate governance risks at 1011778 B.C. is ...,9340,1011778 BC ULC,1.500872,https://www.moodys.com/uploadpage/Mco Document...
1,1,278,2,2,Research: Rating Action: Moody's downgrades Ta...,"sustainability. diversity, equity & inclusion....",11179,1011778 BC ULC,1.591902,https://www.moodys.com/uploadpage/Mco Document...
2,2,277,1,1,"Research: Rating Action: Moody's places 1,721 ...","3. To the extent permitted by law, Moody’s and...",12366,1011778 BC ULC,1.643596,https://www.moodys.com/uploadpage/Mco Document...
3,3,280,4,1,"3D Systems Announces Launch of Environmental, ...","3D Systems Announces Launch of Environmental, ...",6748,3D Systems Corp,1.323604,https://www.3dsystems.com/sites/default/files/...
4,4,287,11,8,3D Systems Reports 2012 Results | 3D Systems,3D Systems Reports 2012 Results. Delivers Reco...,9084,3D Systems Corp,1.486445,https://www.3dsystems.com/sites/default/files/...
...,...,...,...,...,...,...,...,...,...,...
4362,5602,15047,14956,1,Sustainability strategy and targets - thyssenk...,Sustainability strategy and targets. Sustainab...,4158,thyssenkrupp AG,1.151930,https://ucpcdn.thyssenkrupp.com/_binary/UCPthy...
4362,5602,15047,14956,1,Sustainability strategy and targets - thyssenk...,Sustainability strategy and targets. Sustainab...,4158,thyssenkrupp AG,1.151930,https://ucpcdn.thyssenkrupp.com/_binary/UCPthy...
4362,5602,15047,14956,1,Sustainability strategy and targets - thyssenk...,Sustainability strategy and targets. Sustainab...,4158,thyssenkrupp AG,1.151930,https://ucpcdn.thyssenkrupp.com/_binary/UCPthy...
4364,5604,15049,14958,3,Dialogue & Commitment - thyssenkrupp,thyssenkrupp regards the ideas and requirement...,5053,thyssenkrupp AG,1.207290,https://ucpcdn.thyssenkrupp.com/_binary/UCPthy...


In [30]:
df_pdf = pd.concat([df2,news_data])[["company", "site","summary","pdf_url","rank"]]
df_pdf.to_csv("./wiki_data/sus_reports_pdf_url2.csv")

In [ ]:
###########################################

In [1]:
import sys
sys.path.append("/home/qingzhi/.conda/envs/test2/lib/python3.9/site-packages")
import pandas as pd
df_pdf = pd.read_csv("./wiki_data/sus_reports_pdf_url2.csv")
df_pdf

,Unnamed: 0,company,site,summary,pdf_url,rank
0,21,Oversea-Chinese Banking Corp Ltd,Sustainability Report 2020 Sustainability Repo...,Sustainability Report Scope This report covers...,https://www.ocbc.com/iwov-resources/sg/ocbc/gb...,2
1,23,Oversea-Chinese Banking Corp Ltd,Sustainability Report 2019 - OCBC,This report covers the operations of OCBC Grou...,https://www.ocbc.com/iwov-resources/sg/ocbc/gb...,4
2,24,Oversea-Chinese Banking Corp Ltd,Sustainability Report 2018 - OCBC,56 OCBC Annual Report 2018 Sustainability Repo...,https://www.ocbc.com/assets/pdf/media/2019/sus...,5
3,26,Oversea-Chinese Banking Corp Ltd,SUSTAINABILITY ABOUT THE REPORT REPORT - OCBC,SUSTAINABILITY REPORT OUR ESG FACTORS GRI REPO...,https://www.ocbc.com/iwov-resources/sg/ocbc/gb...,7
4,29,China Traditional Chinese Medicine Holdings Co...,"Environmental, Social and t 2017 - China TCM","Environmental, Social and Governance Report 20...",http://www.china-tcm.com.cn/UploadFiles/Files/...,1
...,...,...,...,...,...,...
20175,4362,thyssenkrupp AG,Sustainability strategy and targets - thyssenk...,Sustainability strategy and targets. Sustainab...,https://ucpcdn.thyssenkrupp.com/_binary/UCPthy...,4158
20176,4362,thyssenkrupp AG,Sustainability strategy and targets - thyssenk...,Sustainability strategy and targets. Sustainab...,https://ucpcdn.thyssenkrupp.com/_binary/UCPthy...,4158
20177,4362,thyssenkrupp AG,Sustainability strategy and targets - thyssenk...,Sustainability strategy and targets. Sustainab...,https://ucpcdn.thyssenkrupp.com/_binary/UCPthy...,4158
20178,4364,thyssenkrupp AG,Dialogue & Commitment - thyssenkrupp,thyssenkrupp regards the ideas and requirement...,https://ucpcdn.thyssenkrupp.com/_binary/UCPthy...,5053


In [2]:
# len(df_pdf.drop_duplicates(subset=["company", "pdf_url"], keep="first").company.unique())
# len(df_pdf.company.unique())
df_pdf = df_pdf.drop_duplicates(subset=["company","pdf_url"], keep="first")

In [3]:

%load_ext autoreload
%autoreload 2
# Used to import data from local.
import pandas as pd

# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer

# Used to create and store the Faiss index.
import faiss
import numpy as np
import pickle
from pathlib import Path

# Used to do vector searches and display the results.
from vector_engine.vector_engine.utils import vector_search, id2details


def get_id(x):
    return int(x[1:])
    
# df["id"] = df["wikidata_id"].apply(get_id)
# df = df_rank.loc[4000000:]
df_pdf["id"] = range(len(df_pdf))

# Instantiate the sentence-level DistilBERT
# model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
# model = SentenceTransformer('all-distilroberta-v1 ')
# model = SentenceTransformer('paraphrase-MiniLM-L3-v2')
model = SentenceTransformer('all-mpnet-base-v2')

# Check if GPU is available and use it
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

# Convert abstracts to vectors
embeddings = model.encode(df_pdf.pdf_url.tolist(), show_progress_bar=True)

print("saving embeddings")
with open('./data/embeddings_cleaned.pkl','wb') as f:
    pickle.dump(embeddings, f, protocol=pickle.HIGHEST_PROTOCOL)
    
# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, 
                   df_pdf["id"].values.astype(np.int64))

print(f"Number of vectors in the Faiss index: {index.ntotal}")

# user_query = "sustainability report 2021, sustainability report 2020, sustainability report 2019, sustainability report 2018, ESG report, climate, social, governance"
user_query = "sustainability report 2021, sustainabilityreport2021, sustainability report 2020, sustainabilityreport2020, sustainability report 2019, sustainabilityreport2019"
# user_query = "2021, 2020, 2019, sustainability report"
# user_query = "sustainability report 2021"
D, I = vector_search([user_query], model, index, index.ntotal)
df_temp = df_pdf.set_index("id").loc[I[0]]
df_temp["rank"] = range(len(df_temp))
df_temp["relevance_score"] = D.flatten().tolist()
df_temp

/tmp/ipykernel_3332/70900274.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pdf["id"] = range(len(df_pdf))


cuda:0


Batches:   0%|          | 0/494 [00:00<?, ?it/s]

saving embeddings
Number of vectors in the Faiss index: 15790


,Unnamed: 0,company,site,summary,pdf_url,rank,relevance_score
id,,,,,,,
12209,3080,Raiffeisen Bank International AG,Sustainability Report - rbinternational.com,We invite you to send us your questions and su...,https://www.rbinternational.com/en/sustainabil...,0,0.670837
12216,3080,Raiffeisen Bank International AG,Sustainability Report - rbinternational.com,We invite you to send us your questions and su...,https://www.rbinternational.com/en/sustainabil...,1,0.672520
12211,3080,Raiffeisen Bank International AG,Sustainability Report - rbinternational.com,We invite you to send us your questions and su...,https://www.rbinternational.com/en/sustainabil...,2,0.675059
12213,3080,Raiffeisen Bank International AG,Sustainability Report - rbinternational.com,We invite you to send us your questions and su...,https://www.rbinternational.com/en/sustainabil...,3,0.675584
1219,5714,Enterprise Products Partners LP,2019-2020 Sustainability Report - Enterprise P...,We are pleased to have the opportunity to publ...,http://www.enterpriseproducts.com/documents?do...,4,0.684110
...,...,...,...,...,...,...,...
12200,3069,REN - Redes Energeticas Nacionais SGPS SA,REN - Code of Conduct,Code of Conduct. The Code of Conduct sets out ...,https://www.ren.pt/files/2019-07/2019-07-02093...,15785,2.023610
12199,3069,REN - Redes Energeticas Nacionais SGPS SA,REN - Code of Conduct,Code of Conduct. The Code of Conduct sets out ...,https://www.ren.pt/files/2019-07/2019-07-02093...,15786,2.027741
12197,3069,REN - Redes Energeticas Nacionais SGPS SA,REN - Code of Conduct,Code of Conduct. The Code of Conduct sets out ...,https://www.ren.pt/files/2019-07/2019-07-02093...,15787,2.072670


In [4]:
df_pdf = df_temp.groupby("company").head(3)
# df_pdf[df_pdf.pdf_url.str.contains("2011")]
df_pdf.to_csv("./wiki_data/report_temp.csv")

In [1]:
import sys
sys.path.append("/home/qingzhi/.conda/envs/test/lib/python3.9/site-packages")
import pandas as pd
df_pdf = pd.read_csv("./wiki_data/report_temp.csv")
df_pdf

,id,Unnamed: 0,company,site,summary,pdf_url,rank,relevance_score
0,12209,3080,Raiffeisen Bank International AG,Sustainability Report - rbinternational.com,We invite you to send us your questions and su...,https://www.rbinternational.com/en/sustainabil...,0,0.670837
1,12216,3080,Raiffeisen Bank International AG,Sustainability Report - rbinternational.com,We invite you to send us your questions and su...,https://www.rbinternational.com/en/sustainabil...,1,0.672520
2,12211,3080,Raiffeisen Bank International AG,Sustainability Report - rbinternational.com,We invite you to send us your questions and su...,https://www.rbinternational.com/en/sustainabil...,2,0.675059
3,1219,5714,Enterprise Products Partners LP,2019-2020 Sustainability Report - Enterprise P...,We are pleased to have the opportunity to publ...,http://www.enterpriseproducts.com/documents?do...,4,0.684110
4,683,2922,Calavo Growers Inc,2019 SUSTAINABILITY HIGHLIGHTS REPORT - Calavo...,"CALAVO GROWERS, INC. 2019 SUSTAINABILITY HIGHL...",http://www.calavo.com/pdfs/2019-Sustainability...,5,0.693384
...,...,...,...,...,...,...,...,...
3907,3588,130,Advantest Corp,Environment | Sustainability | ADVANTEST CORPO...,Environment. Mitigation of Climate Change (Pre...,https://www.advantest.com/about/pdf/E_03_IAR20...,15752,1.939996
3908,3585,122,Advanced Lithium Electrochemistry Cayman Co Ltd,Trimble Navigation Limited - Relationship Science,Sustainable Investing for the Long Term Genera...,https://go.relsci.com/l/19262/2015-03-05/2p941...,15763,1.948292
3909,12198,3069,REN - Redes Energeticas Nacionais SGPS SA,REN - Code of Conduct,Code of Conduct. The Code of Conduct sets out ...,https://www.ren.pt/files/2019-07/2019-07-02093...,15782,2.011616
3910,12201,3069,REN - Redes Energeticas Nacionais SGPS SA,REN - Code of Conduct,Code of Conduct. The Code of Conduct sets out ...,https://www.ren.pt/files/2019-07/2019-07-02093...,15784,2.021116


In [3]:
import sys
sys.setrecursionlimit(50000)

import grequests

class Test(object):
    def __init__(self, urls):
        self.urls = urls

    def exception(self, request, exception):
        print("Problem: {}: {}".format(request.url, exception))

    def async1(self):
        results = grequests.map((grequests.get(u, timeout=60, stream=False) for u in self.urls), exception_handler=self.exception, size=5)
        # print(results)
        return results
    
# test = Test(news_data.url.values.tolist()[0:100])
# results = test.async1()
# results
from threading import Thread
import functools

def timeout(timeout):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            res = [Exception('function [%s] timeout [%s seconds] exceeded!' % (func.__name__, timeout))]
            def newFunc():
                try:
                    res[0] = func(*args, **kwargs)
                except Exception as e:
                    res[0] = e
            t = Thread(target=newFunc)
            t.daemon = True
            try:
                t.start()
                t.join(timeout)
            except Exception as je:
                print ('error starting thread')
                raise je
            ret = res[0]
            if isinstance(ret, BaseException):
                raise ret
            return ret
        return wrapper
    return deco

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.parse import urljoin

from PyPDF2 import PdfFileReader
from io import BytesIO
@timeout(10)
def process_pdf(req):
    open_pdf_file = BytesIO(req.content)
    req.close()
    pdf = PdfFileReader(open_pdf_file, strict=False)
    text = [pdf.getPage(i).extractText() for i in range(0, pdf.getNumPages())]
    return " ".join(text)


from tqdm import tqdm
tqdm.pandas()
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

content_list = []
for i in tqdm(list(chunks(range(0, len(df_pdf.pdf_url.values)), 10))):
    urls = df_pdf.pdf_url.values[i]
    test = Test(urls)
    results = test.async1()
    # for req, url in tqdm(zip(results, urls), total=len(urls), leave=False):
    for req, url in tqdm(zip(results, urls), total=len(urls), leave=False):
        
        if req == None:
            content_list.append(None)
        elif req.status_code == 200:
            try:
                content_list.append(process_pdf(req))
                print(url)
            except:
                content_list.append(None)
        else:
            content_list.append(None)
            
# news_data["pdf_list"] = content_list
# content_list

  0%|                                                   | 0/152 [00:00<?, ?it/s]

Problem: https://www.rbinternational.com/en/sustainability/sustainability-report/_jcr_content/root/responsivegrid/tabaccordioncontaine/tabAccordionElements/tabaccordionelement__1228007684/items/downloadlist.download.html/0/Sustainability%20Report%202020.pdf: HTTPSConnectionPool(host='www.rbinternational.com', port=443): Read timed out.



 20%|████████▊                                   | 2/10 [00:00<00:01,  5.26it/s]

http://www.calavo.com/pdfs/2019-Sustainability-Highlights-Report-FINAL.pdf



 30%|█████████████▏                              | 3/10 [00:01<00:03,  2.18it/s]

...

https://www.iqvia.com/-/media/iqvia/pdfs/about-us/2019-sustainability-report.pdf?la=en&hash=249BDD7DBFD1AAF9714AAF63037806DB



  7%|██▉                                     | 11/152 [13:22<2:27:15, 62.66s/it]

https://www.eliagroup.eu/-/media/project/elia/shared/documents/elia-group/publications/annual-reports/2021/en/20210415_elia-group-sustainability-report-2020_en.pdf


KeyboardInterrupt
2022-03-02T20:53:46Z
  7%|██▉                                     | 11/152 [15:17<3:16:06, 83.45s/it]


KeyboardInterrupt: 

In [ ]:
df_pdf["pdf_content"] = content_list
df_pdf = df_pdf.dropna()
df_pdf.to_csv("wiki_data/sus_reports_content.tsv", sep="\t")

In [ ]:
df_pdf["pdf_content"] = df_pdf["pdf_content"].apply(lambda x: np.nan if x==np.nan else str(x).encode('utf-8', 'replace').decode('utf-8'))

In [38]:
############################sus web
import sys
sys.path.append("/home/qingzhi/.conda/envs/test2/lib/python3.9/site-packages")

import pandas as pd
df = pd.read_csv("./wiki_data/sus_reports_url.csv")
df_pdf = df[~df.url.str.lower().str.contains("pdf")]


%load_ext autoreload
%autoreload 2
# Used to import data from local.
import pandas as pd

# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer

# Used to create and store the Faiss index.
import faiss
import numpy as np
import pickle
from pathlib import Path

# Used to do vector searches and display the results.
from vector_engine.vector_engine.utils import vector_search, id2details


def get_id(x):
    return int(x[1:])
    
# df["id"] = df["wikidata_id"].apply(get_id)
# df = df_rank.loc[4000000:]
df_pdf["id"] = range(len(df_pdf))

# Instantiate the sentence-level DistilBERT
# model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
# model = SentenceTransformer('all-distilroberta-v1 ')
# model = SentenceTransformer('paraphrase-MiniLM-L3-v2')
model = SentenceTransformer('all-mpnet-base-v2')

# Check if GPU is available and use it
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

# Convert abstracts to vectors
embeddings = model.encode(df_pdf.url.tolist(), show_progress_bar=True)

print("saving embeddings")
with open('./data/embeddings_cleaned.pkl','wb') as f:
    pickle.dump(embeddings, f, protocol=pickle.HIGHEST_PROTOCOL)
    
# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, 
                   df_pdf["id"].values.astype(np.int64))

print(f"Number of vectors in the Faiss index: {index.ntotal}")

# user_query = "sustainability report 2021, sustainability report 2020, sustainability report 2019, sustainability report 2018, ESG report, climate, social, governance"
# user_query = "sustainability report 2021, sustainabilityreport2021, sustainability report 2020, sustainabilityreport2020, sustainability report 2019, sustainabilityreport2019"
# user_query = "2021, 2020, 2019, sustainability report"
user_query = "sustainability, SDG, ESG, climate, social, governance, environment"
D, I = vector_search([user_query], model, index, index.ntotal)
df_temp = df_pdf.set_index("id").loc[I[0]]
df_temp["rank"] = range(len(df_temp))
df_temp["relevance_score"] = D.flatten().tolist()
df_temp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/tmp/ipykernel_98148/312406442.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pdf["id"] = range(len(df_pdf))


cuda:0


Batches:   0%|          | 0/431 [00:00<?, ?it/s]

saving embeddings
Number of vectors in the Faiss index: 13792


,Unnamed: 0,Unnamed: 0.1,site,summary,url,rank,company,relevance_score
id,,,,,,,,
6407,7878,1,INTERNATIONAL PAPER,Sustainability is integral to all Internationa...,http://www.internationalpaper.com/apps/sustain...,0,International Paper Co,0.639222
2420,2895,5,Reaffirmed ESG Governance - Cascades,Reaffirmed ESG Governance. With the growing in...,https://www.cascades.com/en/sustainable-develo...,1,Cascades Inc,0.641328
4958,6089,4,"Environmental, Social & Governance - Investor ...","Here at FSFL, environmental, social and govern...",https://fsfl.foresightgroup.eu/sustainability-...,2,Foresight Solar Fund Ltd,0.649288
2980,3624,1,CDL | CDL Integrated Sustainability Report 202...,"Singapore, 15 April 2021 – City Developments L...",https://cdl.com.sg/newsroom/cdl-integrated-sus...,3,City Developments Ltd,0.654088
11999,14801,1,Acting responsibly on the entire value chain -...,Tereos has opted for sustainable agriculture b...,https://tereos.com/en/sustainable-development/,4,Tereos SCA,0.662111
...,...,...,...,...,...,...,...,...
6330,7784,7,United States - English - Ingredion,"In a world that rewards the agile, inventive a...",https://www.ingredion.com/,13787,Ingredion Inc,2.076139
6726,8281,7,Privacy Policy,Corporate Office; Jubilant Pharma Holdings Inc...,http://www.jubilantpharma.com/privacy-policy,13788,Jubilant Pharma Ltd/SG,2.087318
232,52,1,Aalberts - mission critical technologies,Aalberts integrated piping systems has introdu...,https://aalberts.com/,13789,Aalberts NV,2.102955


In [46]:
news_data = df_temp.groupby("company").head(3)
news_data.to_csv("temp2.csv")

In [2]:

import sys
sys.path.append("/home/qingzhi/.conda/envs/test/lib/python3.9/site-packages")
import pandas as pd
news_data = pd.read_csv("temp2.csv")
# news_data

In [5]:
import grequests

class Test(object):
    def __init__(self, urls):
        self.urls = urls

    def exception(self, request, exception):
        print("Problem: {}: {}".format(request.url, exception))

    def async1(self):
        results = grequests.map((grequests.get(u, timeout=10, stream=False) for u in self.urls), exception_handler=self.exception, size=10)
        # print(results)
        return results
    
# test = Test(news_data.url.values.tolist()[0:100])
# results = test.async1()
# results

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.parse import urljoin
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

content_list = []
for i in tqdm(list(chunks(range(0, len(news_data.url.values)), 20))):
    urls = news_data.url.values[i]
    test = Test(urls)
    results = test.async1()
    # for req, url in tqdm(zip(results, urls), total=len(urls), leave=False):
    for req, url in zip(results, urls):
        if req == None:
            content_list.append(None)
        elif req.status_code == 200:
            soup = BeautifulSoup(req.content, 'html.parser', from_encoding="iso-8859-1")
            raw_text = []
            for element in soup.findAll(text=True):
                raw_text.append(element.getText())
            content_list.append(" ".join(raw_text))
        else:
            content_list.append(None)
            
# news_data["pdf_list"] = content_list
# content_list

  0%|                                                   | 0/273 [00:00<?, ?it/s]

Problem: http://www.internationalpaper.com/apps/sustainabilityreport2013/our-approach-to-sustainability/sustainability-governance.html: HTTPSConnectionPool(host='www.internationalpaper.com', port=443): Max retries exceeded with url: /apps/sustainabilityreport2013/our-approach-to-sustainability/sustainability-governance.html (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)')))


  1%|▎                                          | 2/273 [00:13<31:45,  7.03s/it]

Problem: https://www.cosmote.gr/cs/otegroup/en/sustainability_governance.html: Exceeded 30 redirects.
Problem: https://www.tdk.com/en/sustainability2021/governance/index.html: HTTPSConnectionPool(host='www.tdk.com', port=443): Read timed out. (read timeout=10)


  1%|▋                                          | 4/273 [00:44<51:23, 11.46s/it]

...

Problem: https://www.dish.com/dish-cares/: HTTPSConnectionPool(host='www.dish.com', port=443): Read timed out. (read timeout=10)
Problem: https://www.parker.com/portal/site/PARKER/menuitem.7322a3ce19c3a730b5170b9d237ad1ca/?vgnextoid=210c2e1209a6e210VgnVCM10000048021dacRCRD&vgnextfmt=EN&vgnextitem=1a18afb71e7ea710VgnVCM100000e6651dacRCRD&vgnextcat=News%20Release%20Details: HTTPSConnectionPool(host='www.parker.com', port=443): Read timed out. (read timeout=10)


 99%|████████████████████████████████████████▋| 271/273 [38:05<00:21, 10.55s/it]

Problem: http://www.fdgev.com/en/news_detail/368: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Problem: https://www.dish.com/programming/packages/: HTTPSConnectionPool(host='www.dish.com', port=443): Read timed out. (read timeout=10)


100%|█████████████████████████████████████████| 273/273 [38:26<00:00,  8.45s/it]

Problem: https://www.dish.com/: HTTPSConnectionPool(host='www.dish.com', port=443): Read timed out. (read timeout=10)


In [6]:
news_data["web_content"] = content_list
news_data = news_data.dropna()
news_data.to_csv("./wiki_data/sus_web_content.csv", sep='\t')

In [15]:
##########################################wikipedia
news_data = pd.read_csv("./wiki_data/wiki_url_final.csv",sep="\t")
from urllib.parse import urlparse
from tqdm import tqdm

tqdm.pandas()
def process_url(url):
    p = urlparse(url)
    return "https://"+p.netloc+"/w/index.php?action=raw&title="+p.path.split("/")[-1]

news_data["new_url"] = news_data["url"].progress_apply(process_url)

100%|███████████████████████████████████| 1908/1908 [00:00<00:00, 230023.05it/s]


In [11]:
# pd.read_csv("./wiki_data/sus_web_content.csv", sep='\t')

In [20]:
import grequests

class Test(object):
    def __init__(self, urls):
        self.urls = urls

    def exception(self, request, exception):
        print("Problem: {}: {}".format(request.url, exception))

    def async1(self):
        results = grequests.map((grequests.get(u, timeout=10, stream=False) for u in self.urls), exception_handler=self.exception, size=20)
        # print(results)
        return results
    
# test = Test(news_data.url.values.tolist()[0:100])
# results = test.async1()
# results

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.parse import urljoin
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

content_list = []
for i in tqdm(list(chunks(range(0, len(news_data.new_url.values)), 20))):
    urls = news_data.new_url.values[i]
    test = Test(urls)
    results = test.async1()
    # for req, url in tqdm(zip(results, urls), total=len(urls), leave=False):
    for req, url in zip(results, urls):
        if req == None:
            content_list.append(None)
        elif req.status_code == 200:
            soup = BeautifulSoup(req.content, 'html.parser', from_encoding="iso-8859-1")
            raw_text = []
            for element in soup.findAll(text=True):
                raw_text.append(element.getText())
            content_list.append(" ".join(raw_text))
        else:
            content_list.append(None)
            
# news_data["pdf_list"] = content_list
# content_list

 64%|███████████████████████████▎               | 61/96 [00:52<01:58,  3.40s/it]

Problem: https://en.wikipedia.org/w/index.php?action=raw&title=Pearson_plc: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Read timed out. (read timeout=10)


 96%|█████████████████████████████████████████▏ | 92/96 [01:23<00:12,  3.09s/it]

Problem: https://www.dsnl.co.kr/w/index.php?action=raw&title=about-overview.html: HTTPSConnectionPool(host='www.dsnl.co.kr', port=443): Max retries exceeded with url: /w/index.php?action=raw&title=about-overview.html (Caused by SSLError(SSLCertVerificationError("hostname 'www.dsnl.co.kr' doesn't match either of '*.oneduksan.com', 'oneduksan.com'")))


 99%|██████████████████████████████████████████▌| 95/96 [01:43<00:05,  5.71s/it]

Problem: https://summit-materials.com/w/index.php?action=raw&title=: HTTPSConnectionPool(host='summit-materials.com', port=443): Max retries exceeded with url: /w/index.php?action=raw&title= (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f7d8d3bc550>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))


100%|███████████████████████████████████████████| 96/96 [01:43<00:00,  1.08s/it]


In [24]:
news_data["wiki_content"] = content_list
news_data = news_data.dropna()
news_data.to_csv("./wiki_data/wiki_content.csv", sep="\t")